In [1]:
%load_ext autoreload
%autoreload 2

In [152]:
%matplotlib inline
import re
import time
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', -1)

In [230]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Generate Summaries from Complaint Narratives

In [ ]:
df.Product.unique()

In [ ]:
df[df.Product == 'Credit reporting, credit repair services, or other personal consumer reports'].Issue.value_counts()

In [308]:
credit_report = df[df.Product == 'Credit reporting, credit repair services, or other personal consumer reports']
credit_report = credit_report[credit_report.Issue == 'Incorrect information on your report']

In [241]:
df = pd.read_csv('../data/product_merged.csv', usecols=['Complaint ID', 'Product', 'Issue',
                                                       'Consumer complaint narrative'])
subset = df[df.Product == 'Credit card or prepaid card']

[nltk_data] Downloading package punkt to /Users/sueliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [242]:
import os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [243]:
from preprocess.clean_and_tokenize import tokenize_sentences, clean_and_tokenize

In [244]:
from scripts.text_summarize_textrank import gen_sent_vectors, compute_similarity_matrix, rank_sentences, gen_top_n_sentences

In [245]:
sentences = tokenize_sentences(subset['Consumer complaint narrative'])

In [259]:
clean_sentences = clean_and_tokenize(sentences)

In [9]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [260]:
import pickle
data = {'sentences': sentences, 'clean_sentences': clean_sentences, 'embeddings': word_embeddings, 
        'complaint_id': subset['Complaint ID'].values}
with open('../output/credit_card_sentences.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [261]:
clean_sentences

[['unauthorized transfer american express serve card different people unknown transfer authorize',
  '',
  'time transfer occur notify american express serve change password pin secret question',
  'money transfer account follow security change',
  'keep tell fraud protect card bank day resolve dispute',
  'call frequently request update gradually frustrated tell day resolve dispute victim identity theft card credit refuse speak un authorized transaction daily call request update',
  'call ask speak individual fraud dept inform accept call',
  'upset request send email update',
  'hour american express serve send email state close account state violate contract authorize transfer',
  'american express serve refuse permit speak representative matter',
  'health issue utility shut incur late fee week deposit money',
  'file police report transaction encourage retrieve money',
  'frustrated'],
 ['notice money available cardcom prepay card go tablet log notice bunch charge',
  'immediately

In [262]:
summarized = [gen_top_n_sentences(sentences[idx], clean_sentences[idx], word_embeddings) for idx in range(len(subset))]

In [263]:
cleaned_summarized = [' '.join(clean_sentences[idx]) for idx in range(len(subset))]
subset['summarized'] = summarized
subset['cleaned_summarized'] = cleaned_summarized

In [264]:
subset['summarized'].head()

183725    I called frequently requesting updates gradually becoming more frustrated now being told they have 45 days to resolve the dispute as I was " A victim of Identity Theft '' .The card credited me back the {$780.00} only & REFUSES to speak to me about the other 2 un authorized transactions after daily calls requesting updates. Within an hour, American Express Serve sent me an e-mail stating they closed out the account stating " I violated the contract '' & that " I authorized the transfers ''. Each time these transfers occurred, I notified American Express Serve, where they changed my password, PIN & secret question. Money was still transferred from my account following the security changes. I kept being told that I was 100 % Fraud Protected with my Card but the bank had up 10 days to resolve the dispute.
183726    I ca n't believe I was robbed and is n't standing behind their word of taking care of me ... I was sooooo angry, so I started calling the merchants to try to fix my pr

In [108]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ",".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [266]:
subset.to_csv('../data/credit_card_with_summaries.csv', index=False)

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


In [136]:
subset = pd.read_csv('../data/credit_card_with_summaries.csv')

In [267]:
import re
remove_list = ["american express", "wells fargo", "fargo", "bank of america", 
               "bank america", "ocwen", "equifax", "morgan", "chase", "citibank", 
               "navient", "nationstar", "capital", "citi", "amex" "capital one", "synchrony", "costco", "nt"]
remove = '|'.join(remove_list)
pattern = re.compile(r'\b('+remove+r')\b', flags=re.IGNORECASE)
remove_companies = [pattern.sub("", s) for s in subset['cleaned_summarized'].values]

In [268]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

X = vectorizer.fit_transform(remove_companies)

## Application of NMF

In [269]:
from sklearn.decomposition import NMF

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (X.shape[0], X.shape[1]))
t0 = time.time()
nmf = NMF(n_components=12, random_state=1,
          alpha=.1, l1_ratio=.5).fit(X)

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=46671 and n_features=5000...


In [270]:
print("done in %0.3fs." % (time.time() - t0))
n_top_words = 20
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

done in 8.066s.

Topics in NMF model (Frobenius norm):
Topic #0: tell,call,say,receive,send,number,ask,not,money,phone,time,service,customer,day,speak,email,check,card,customer service,letter
Topic #1: statement,statement thought,believe payment,happen believe,shock review,sure happen,thought,report find,review credit,shock,late,find late,receive statement,payment receive,payment date,thought monthly,late payment,date day,payment,sure
Topic #2: report,credit report,credit,remove,inquiry,debt,information,reporting,report credit,bureaus,agency,account,identity,file,credit bureaus,credit reporting,letter,theft,open,remove credit
Topic #3: charge,dispute,merchant,fraudulent,transaction,fraud,dispute charge,fraudulent charge,provide,claim,company,refund,purchase,file,contact,unauthorized,discover,item,charge credit,charge account
Topic #4: card,credit card,credit,card company,apply,new,company,bank,use,new card,open,purchase,card account,cancel,year,application,well,apply credit,activate,is

In [227]:
nmf_topics = nmf.transform(X)
nmf_topics = pd.DataFrame(nmf_topics)

nmf_predicted_topics = pd.merge(subset.reset_index().reset_index(), nmf_topics.reset_index(),  left_on='level_0', right_on='index')
nmf_predicted_topics.drop(columns=['level_0', 'index_x', 'index_y'], inplace=True)

nmf_predicted_topics['most_likely'] = np.argmax(nmf_topics.values, axis=1)

In [228]:
nmf_predicted_topics['most_likely'].value_counts()

0     9679
3     6969
8     5182
9     4830
6     4509
4     3630
5     3028
7     2841
1     2114
11    1791
10    1353
2     745 
Name: most_likely, dtype: int64

In [200]:
nmf_predicted_topics.Issue.value_counts()

Problem with a purchase shown on your statement                                     5767
Other features, terms, or problems                                                  3792
Fees or interest                                                                    3601
Billing disputes                                                                    3102
Problem when making payments                                                        2741
Getting a credit card                                                               2157
Closing your account                                                                2104
Advertising and marketing, including promotional offers                             2033
Other                                                                               1940
Identity theft / Fraud / Embezzlement                                               1723
Closing/Cancelling account                                                          1440
Trouble using your ca

In [291]:
nmf_predicted_topics[nmf_predicted_topics.most_likely == 11]['Issue'].value_counts()

Other features, terms, or problems                                                  278
Fees or interest                                                                    226
Balance transfer                                                                    189
Advertising and marketing, including promotional offers                             141
Problem when making payments                                                        103
Problem with a purchase shown on your statement                                     101
Billing disputes                                                                    87 
Other                                                                               64 
Payoff process                                                                      57 
Billing statement                                                                   51 
Closing your account                                                                45 
APR or interest rate            

In [229]:
nmf_predicted_topics[['Issue', 'summarized', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 'most_likely']]

,Issue,summarized,0,1,2,3,4,5,6,7,8,9,10,11,most_likely
0,Unauthorized transactions/trans. issues,"I called frequently requesting updates gradually becoming more frustrated now being told they have 45 days to resolve the dispute as I was "" A victim of Identity Theft '' .The card credited me back the {$780.00} only & REFUSES to speak to me about the other 2 un authorized transactions after daily calls requesting updates. Within an hour, American Express Serve sent me an e-mail stating they closed out the account stating "" I violated the contract '' & that "" I authorized the transfers ''. I called on & asked to speak with an individual in the Fraud Dept & was informed they DO NOT ACCEPT CALLS. Each time these transfers occurred, I notified American Express Serve, where they changed my password, PIN & secret question. I kept being told that I was 100 % Fraud Protected with my Card but the bank had up 10 days to resolve the dispute.",0.014917,0.000000,0.000000,0.000000,0.000000,0.000000,0.006071,0.008541,0.008857,0.001297,0.0,0.001351,0
1,Unauthorized transactions/trans. issues,"I was sooooo angry, so I started calling the merchants to try to fix my problem from their end and much to my surprise EVERY one of those merchants told me that they had already refunded the claim and that I should call my card company. I ca n't believe I was robbed and is n't standing behind their word of taking care of me ... After waiting a month to hear back from card.com, they not only did n't give me my money back for the unauthorized charges, they re-charged me for one of those unauthorized charges. On I noticed that the money that should be available I my card.com prepaid card was not, went to my tablet, logged on and noticed a whole bunch of charges that were not made by me. Well I did and not only did they avoid the questions of where is the money now?",0.018893,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025590,0.005262,0.000000,0.0,0.000000,7
2,"Advertising, marketing or disclosures","I was under the impression that this practice was illegal, as it made the recipients of these cards easy targets for identity theft. Net Spend sent me a card for which I did not apply. They charge consumers a ridiculous amount of fees for using their own money. I have never had any contact with Net Spend, and until about minutes ago, when I opened some mail, was completely unaware of their existence. It is negligent on their part to send out a card, to my home, that could so easily be activated by someone other than myself.",0.008410,0.001056,0.000000,0.005108,0.000560,0.004870,0.000000,0.003293,0.002150,0.000000,0.0,0.000000,0
3,Fraud or scam,I had a secured card with Wells Fargo for {$2000.00} which I deposited. Then they paid out a {$2500.00} fee to disputed account and after many attempts to refund my scammed money the bank refused to issue credit but instead charged me the money and used my secured money. It 's now being reported to the credit companies as a negative account for - {$2500.00}. This is fraud and they need to be investigated.,0.006685,0.010365,0.000000,0.006300,0.000000,0.000000,0.009153,0.004316,0.003237,0.000000,0.0,0.000000,1
4,Unauthorized transactions/trans. issues,"I called this company SEVERAL TIMES as to why my claim was denied and they flat out refuse to tell me or send me the documentation that I requested for as proof as to why I lost the dispute, WHICH IS ILLEGAL. The reason why I feel this way is because they know my ethnicity and the rep on the phone basically admitted it and believes that there is no reason why my claim should be denied and my account be closed. If their company can not speak well, then there is going to be a mix up of info, which is also how my claims get denied too. I still, to this day, have not even got a chance to respond to the merchants response to the dispute for a rebuttal because I have n't even received the documents that were used in the investigation which again is violating my rights. 

## Applications of LDA

In [303]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

lda_model = LatentDirichletAllocation(n_components=12, random_state=42, n_jobs=-1)
lda_model.fit(X)

# Perplexity
print("Model perplexity: {}".format(lda_model.perplexity(X)))

Model perplexity: 4211.006996187884


In [274]:
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda_model, tf_feature_names, n_top_words)

Topic #0: payment,account,macy,fee,bank,bill,late,pay,charge,late fee,credit,statement,check,good buy,checking,checking account,return,receive,send,bank account
Topic #1: payment,interest,pay,balance,charge,late,fee,month,late fee,statement,credit,card,account,rate,minimum,bill,purchase,date,credit card,interest charge
Topic #2: point,card,offer,reward,bonus,amex,spend,mile,apply,receive,credit,purchase,month,promotion,credit card,tell,account,earn,cash,term
Topic #3: credit,account,report,debt,payment,pay,card,collection,credit report,year,late,credit card,time,agency,balance,company,remove,bankruptcy,discover,close
Topic #4: credit,report,inquiry,credit report,open,card,account,credit card,application,identity,apply,remove,theft,hard,identity theft,paypal,hard inquiry,inquiry credit,apply credit,authorize
Topic #5: fee,annual,annual fee,transfer,balance transfer,balance,card,rate,close,interest,account,charge,credit,credit card,navy,charge annual,interest rate,navy federal,close acco

In [275]:
lda_topics = lda_model.transform(X)
lda_topics = pd.DataFrame(lda_topics)

predicted_topics = pd.merge(subset.reset_index().reset_index(), lda_topics.reset_index(),  left_on='level_0', right_on='index')
predicted_topics.drop(columns=['level_0', 'index_x', 'index_y'], inplace=True)

predicted_topics['most_likely'] = np.argmax(lda_topics.values, axis=1)
predicted_topics['max_prob'] = np.max(lda_topics.values, axis=1)

In [276]:
predicted_topics['most_likely'].value_counts()

8    12593
9    8524 
1    8241 
6    5272 
2    3689 
3    3658 
4    2029 
0    1365 
5    669  
7    631  
Name: most_likely, dtype: int64

In [302]:
predicted_topics[predicted_topics.most_likely==8]['Issue'].value_counts()

Other features, terms, or problems                                                  1296
Problem with a purchase shown on your statement                                     1208
Problem when making payments                                                        780 
Getting a credit card                                                               747 
Closing your account                                                                668 
Identity theft / Fraud / Embezzlement                                               604 
Trouble using your card                                                             499 
Customer service / Customer relations                                               474 
Closing/Cancelling account                                                          474 
Unauthorized transactions/trans. issues                                             462 
Billing disputes                                                                    458 
Other                

In [277]:
predicted_topics[['Issue', 'summarized', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'most_likely', 'max_prob']]

,Issue,summarized,0,1,2,3,4,5,6,7,8,9,most_likely,max_prob
0,Unauthorized transactions/trans. issues,"I called frequently requesting updates gradually becoming more frustrated now being told they have 45 days to resolve the dispute as I was "" A victim of Identity Theft '' .The card credited me back the {$780.00} only & REFUSES to speak to me about the other 2 un authorized transactions after daily calls requesting updates. Within an hour, American Express Serve sent me an e-mail stating they closed out the account stating "" I violated the contract '' & that "" I authorized the transfers ''. Each time these transfers occurred, I notified American Express Serve, where they changed my password, PIN & secret question. Money was still transferred from my account following the security changes. I kept being told that I was 100 % Fraud Protected with my Card but the bank had up 10 days to resolve the dispute.",0.010000,0.009999,0.009997,0.009998,0.010003,0.009998,0.009998,0.009997,0.910010,0.010000,8,0.910010
1,Unauthorized transactions/trans. issues,"I ca n't believe I was robbed and is n't standing behind their word of taking care of me ... I was sooooo angry, so I started calling the merchants to try to fix my problem from their end and much to my surprise EVERY one of those merchants told me that they had already refunded the claim and that I should call my card company. On I noticed that the money that should be available I my card.com prepaid card was not, went to my tablet, logged on and noticed a whole bunch of charges that were not made by me. After waiting a month to hear back from card.com, they not only did n't give me my money back for the unauthorized charges, they re-charged me for one of those unauthorized charges. Well I did and not only did they avoid the questions of where is the money now?",0.011137,0.011139,0.011138,0.011139,0.011138,0.011137,0.011137,0.011137,0.374077,0.536822,9,0.536822
2,"Advertising, marketing or disclosures","I was under the impression that this practice was illegal, as it made the recipients of these cards easy targets for identity theft. Net Spend sent me a card for which I did not apply. They charge consumers a ridiculous amount of fees for using their own money. I have never had any contact with Net Spend, and until about minutes ago, when I opened some mail, was completely unaware of their existence. It is negligent on their part to send out a card, to my home, that could so easily be activated by someone other than myself.",0.014850,0.014853,0.014853,0.014849,0.014861,0.014851,0.014851,0.014846,0.866333,0.014851,8,0.866333
3,Fraud or scam,I had a secured card with Wells Fargo for {$2000.00} which I deposited. Then they paid out a {$2500.00} fee to disputed account and after many attempts to refund my scammed money the bank refused to issue credit but instead charged me the money and used my secured money. It 's now being reported to the credit companies as a negative account for - {$2500.00}. This is fraud and they need to be investigated.,0.015170,0.015165,0.015164,0.863493,0.015166,0.015177,0.015163,0.015160,0.015168,0.015173,3,0.863493
4,Unauthorized transactions/trans. issues,"I called this company SEVERAL TIMES as to why my claim was denied and they flat out refuse to tell me or send me the documentation that I requested for as proof as to why I lost the dispute, WHICH IS ILLEGAL. I called and filed a claim, that of course, got denied. The reason why I feel this way is because they know my ethnicity and the rep on the phone basically admitted it and believes that there is no reason why my claim should be denied and my account be closed. It is time this company stops violating peoples right and listen up! I still, to this day, have not even got a chance to respond to the merchants response to the dispute for a rebuttal because I have n't even received the documents that were used in the investigation which again is violating my rights.",0.009295,0.009295,0.009295,0.009296,0.00929

In [79]:
predicted_topics.to_csv('../output/credit_card_complaint_types.csv', index=False)

In [80]:
predicted_topics.iloc[:100,:].to_csv('../output/credit_card_complaint_types_subset.csv', index=False)

,Product,Issue,Consumer complaint narrative,Complaint ID,summarized,cleaned_summarized,0,1,2,3,4,5,6,7,8,9,most_likely,max_prob
20323,Credit card or prepaid card,Incorrect information on your report,"This home depot card I have had since 2009 and never missed a payment as reflected in my credit report. I went online recently this summer to manage a return and somehow the site made me click a box to proceed which also , in my opinion misrepresented, that I was signing up for online statements. I do not want online statements. I use paper statements each month and I like to pay my bills when they are sent to me. I have a very good track record of doing just that.Because of this , I never received my regular statement and now I show a 30 day late for home depot which is false. I should not have this. I never asked for my statements to be switched. This to me is unfair and not a good practice. I want the 30 day late removed",3345477,"I use paper statements each month and I like to pay my bills when they are sent to me. I went online recently this summer to manage a return and somehow the site made me click a box to proceed which also , in my opinion misrepresented, that I was signing up for online statements. I have a very good track record of doing just that.Because of this , I never received my regular statement and now I show a 30 day late for home depot which is false. I do not want online statements. I never asked for my statements to be switched.",home depot card miss payment reflect credit report go online recently summer manage return site click box proceed opinion misrepresent sign online statement want online statement use paper statement month like pay bill send good track record thatbecause receive regular statement day late home depot false ask statement switch unfair good practice want day late remove,0.013232,0.880921,0.013231,0.013232,0.013231,0.013229,0.013231,0.013233,0.013230,0.013231,1,0.880921
20341,Credit card or prepaid card,Incorrect information on your report,"Made 3 separate purchases in XX/XX/2019, totaling ~ $XXXX. Never received a statement to my home or email. I received an alert from XXXX XXXX on XX/XX/19 that my credit score dropped by 124 points. I went through XXXX XXXX to determine the potential issue as I was stunned by the news (my credit was great previously and has been). I found a note that said 60 day collection Best Buy. I logged into my Best Buy account and was surprised to see charges stemming from XXXX when late fees of $XXXX/month + interest $XXXX. I called immediately and told them I was surprised by all of this as I have not received any statements to my email or to my home indicating there was a balance or any late fees. They told me they emailed me but it could have gone to a junk folder. This is not acceptable. The $XXXX is meaningless to me in terms of dollar amount and could have been easily paid. This is a predatory action that is ridiculous as I cancelled the card while on the phone.",3343958,I called immediately and told them I was surprised by all of this as I have not received any statements to my email or to my home indicating there was a balance or any late fees. I logged into my Best Buy account and was surprised to see charges stemming from when late fees of $ + interest $ I went through to determine the potential issue as I was stunned by the news (my credit was great previously and has been). I found a note that said 60 day collection Best Buy. I received an alert from on that my credit score dropped by 124 points.,separate purchase total receive statement home email receive alert credit score drop point go determine potential issue stun news credit great previously find note say day collection good buy log good buy account surprised charge stem late fee month interest call immediately tell surprised receive statement email home indicate balance late fee tell email go junk folder acceptable meaningless term dollar easily pay predatory action ridiculous cancel card pho

In [125]:
predicted_topics.most_likely.value_counts()

13    14440
3     10249
8      5154
1      5051
10     4697
5      4180
12     1520
0       780
9       244
14      159
4       112
6        34
2        24
11       22
7         5
Name: most_likely, dtype: int64

In [118]:
predicted_topics.iloc[18,:]

Product                                               Credit card or prepaid card
Issue                                                                Adding money
Consumer complaint narrative    On XX/XX/XXXX 2017, American Express made a de...
Complaint ID                                                              2427113
summarized                      Having I know that any adverse action by a ban...
cleaned_summarized              american express decision limit add funds blue...
0                                                                        0.011004
1                                                                       0.0110069
2                                                                        0.011004
3                                                                        0.467044
4                                                                       0.0110038
5                                                                        0.212468
6               

In [57]:
data = {'model': best_lda_model, 'vectorizer': vectorizer}
with open('../output/credit_card_model.pkl', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)